# Imports and Global Variables

In [1]:
from typing import List
from config.endpoints import MAINNET_BASE_URL
from helpers.utility import _make_get_request, _make_post_request
from helius import NFTAPI, NameAPI, BalancesAPI, WebhooksAPI, TransactionsAPI
from dotenv import load_dotenv
from pathlib import Path
import requests
import pandas as pd
import numpy as np
import os
import time
import json
from collections import defaultdict


In [2]:
try:
    ROOT_DIR = Path(__file__).resolve().parent
except NameError:
    ROOT_DIR = Path(os.getcwd()).resolve()

In [3]:
ROOT_DIR

PosixPath('/Users/olaoluwatunmise/Solana-Forensic-Analytics/Solana-Forensic-Analysis-Tool/notebooks')

In [4]:
# Persistent Variables

LAMPORT_SCALE = 1e9

In [5]:
load_dotenv()

False

In [6]:
# class TransactionsAPI:
#     def __init__(self, api_key: str):
#         self.base_url = MAINNET_BASE_URL
#         self.headers = {"Authorization": f"Bearer {api_key}"}
#         self.api_key_query = f"?api-key={api_key}"

#     def get_raw_transactions(
#         self, 
#         accounts: List[str], 
#         start_slot: int=None,
#         end_slot: int=None,
#         start_time: int=None,
#         end_time: int=None,
#         limit: int=None,
#         sort_order: str=None,
#         pagination_token: str=None
#         ):

#         path = "/v1/raw-transactions"
#         url = self.base_url + path + self.api_key_query
#         payload = {
#             "query": {
#                 "accounts": accounts,
#                 "startSlot": start_slot,
#                 "endSlot": end_slot,
#                 "startTime": start_time,
#                 "endTime": end_time
#             },
#             "options": {
#                 "limit": limit,
#                 "sortOrder": sort_order,
#                 "paginationToken": pagination_token
#             }
#         }
#         return _make_post_request(url, payload)

#     def get_parsed_transactions(self, transactions: List[str], commitment: str=None):
#         path = "/v0/transactions"
#         if commitment:
#             url = self.base_url + path + self.api_key_query + "?commitment=commitment"
#         else:
#             url = self.base_url + path + self.api_key_query
#         payload = {
#             "transactions": transactions
#         }
#         return _make_post_request(url, payload)

#     def get_parsed_transaction_history(
#         self, 
#         address: str, 
#         before: str='', 
#         until: str='', 
#         commitment: str='',
#         source: str='',
#         type: str=''
#         ):
        
#         path = f"/v0/addresses/{address}/transactions"
#         params = {
#             "before": before,
#             "until": until,
#             "commitment": commitment,
#             "source": source,
#             "type": type
#         }

#         url = self.base_url + path + self.api_key_query
#         print(f'url: {url}')
#         return _make_get_request(url, params=params)

In [7]:
HELIUS_API_KEY = os.getenv('HELIUS_API_KEY')
VYBE_API_KEY = os.getenv('VYBE_API_KEY')
NANSEN_API_KEY = os.getenv('NANSEN_API_KEY')


In [8]:
use_cache = True
test_address = 'AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU'

# Price Data

In [9]:
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame

In [10]:
client = CryptoHistoricalDataClient()

request_params = CryptoBarsRequest(
    symbol_or_symbols=["BTC/USD", "ETH/USD"],
    timeframe=TimeFrame.Day,
    start="2022-07-01"
)


# Flipside Data (Balances over time)

Here is the Flipside Query; when moving to live app will use the Flipside API to retrieve this dynamically: https://flipsidecrypto.xyz/studio/queries/f72285d7-f3d4-4596-a9cd-8785187005a2

In [11]:
os.getcwd()

'/Users/olaoluwatunmise/Solana-Forensic-Analytics/Solana-Forensic-Analysis-Tool/notebooks'

In [12]:
# os.chdir('..')

In [13]:
balance_df = pd.read_csv('../data/raw/solana_balance_history_test.csv').dropna(how='all')

# Strip BOMs or invisible characters from column names
balance_df.columns = balance_df.columns.str.replace('\ufeff', '', regex=False).str.strip()

# Optionally: remove rows where any column has just a BOM or is empty/whitespace
balance_df = balance_df[~balance_df.apply(lambda row: row.astype(str).str.contains('\ufeff|^\s*$', regex=True)).any(axis=1)]

sol_mask = (
    (balance_df['MINT'] == 'So11111111111111111111111111111111111111111') &
    (balance_df['SYMBOL'].isna())
)

balance_df.loc[sol_mask, 'SYMBOL'] = 'SOL'
balance_df.loc[sol_mask, 'NAME'] = 'Solana'

balance_df['TX_ID'] = balance_df['TX_ID'].str.lower()

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
/var/folders/9f/dspzp56j3pjcftph22hs9gw40000gn/T/ipykernel_99967/2299430333.py:7: SyntaxWarning: invalid escape sequence '\s'
  balance_df = balance_df[~balance_df.apply(lambda row: row.astype(str).str.contains('\ufeff|^\s*$', regex=True)).any(axis=1)]


In [14]:
# Any nan values left should be LP tokens

balance_df[balance_df['SYMBOL'].isna()]

,BLOCK_TIMESTAMP,OWNER,MINT,PRE_BALANCE,BALANCE,TX_ID,SUCCEEDED,SYMBOL,NAME
490,2024-01-27 20:02:00.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,21.284506,0.000000,ag9wy4nng7vgmgvwmbmikudfcgk9sigh9xhbgumq9lcfl8...,True,NaN,NaN
494,2024-01-27 20:00:42.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.000000,21.284506,3ktq7dyxvuoknsnhkhd4wmgcvxgq5cdxm6zpuunfjc1tgb...,True,NaN,NaN
503,2024-01-27 19:56:38.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,38.261386,0.000000,5xblmj697ixeazt6azmd3fxx26ckyuwhma2pmqmpyacdry...,True,NaN,NaN
510,2024-01-27 19:54:37.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.000000,38.261386,2jgmfwybccr51sgxupejvvprrhas1xyxpgtz96drrxnnmb...,True,NaN,NaN
519,2024-01-27 19:52:56.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,12.880316,0.000000,wmkk8qpz8xnk1kw1zaggersbbbegdm6prxwyidvbdcu4wv...,True,NaN,NaN
522,2024-01-27 19:51:51.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,10.000000,12.880316,t1egxx8diqh5ak4ptkny6ajru96dwe2r8yyjraqssj4cxy...,True,NaN,NaN
524,2024-01-27 19:47:10.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.000000,10.000000,deymhggrurmjvatdnhmqauszfxhzrxftxps3e5bgtes7vj...,True,NaN,NaN
539,2024-01-12 00:29:26.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,B9uEZfxJoAsc51BZs6rNs8XtYVu5vfVrhbCuDCJ7mPEE,0.000000,5.000000,qwsnf1ioo2ucggywbzdfa1knav8vujgg7dkwgkazfyp6q6...,True,NaN,NaN
540,2024-01-11 23:02:04.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,5.711749,0.000000,wnz5j6dcgtjmigf8dxvbh72t4ks6bo3kvtsgmbzl7trjdn...,True,NaN,NaN
544,2024-01-11 22:57:58.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.000000,5.711749,5238f2m2jwjbsqsnb5k1l8b6mj5z9d1xs9eeakwzjkpzsf...,True,NaN,NaN


In [15]:
# os.chdir('notebooks')

In [16]:
os.getcwd()

'/Users/olaoluwatunmise/Solana-Forensic-Analytics/Solana-Forensic-Analysis-Tool/notebooks'

# Helius API Calls

In [17]:
def get_all_signatures(account_address, helius_api_key, max_pages=20, limit=100):
    url = f"https://mainnet.helius-rpc.com/?api-key={helius_api_key}"
    collected = []
    before = None

    for _ in range(max_pages):
        payload = {
            "jsonrpc": "2.0",
            "id": "1",
            "method": "getSignaturesForAddress",
            "params": [account_address, {"limit": limit, **({"before": before} if before else {})}]
        }

        response = requests.post(
            url,
            headers={"Content-Type": "application/json"},
            data=json.dumps(payload)
        )

        try:
            data = response.json()
            if "error" in data:
                print(f"❌ Error: {data['error']['message']}")
                break

            batch = data["result"]
            if not batch:
                break

            collected.extend(batch)
            before = batch[-1]["signature"]  # Move to next batch

            time.sleep(0.5)  # Avoid rate-limits

        except Exception as e:
            print(f"Exception: {e}")
            break

    return collected

In [18]:
# signatures = get_all_signatures(test_address, HELIUS_API_KEY)

# signatures_dict = {}

# for sig in signatures:
#     time = sig.get('blockTime')
#     signature_str = sig.get('signature')

#     signatures_dict[signature_str] = time

# signatures_dict

In [19]:
# signatures_array = list(signatures_dict.keys())

In [20]:
def v0_transactions_all(signatures, helius_api_key):
    import time
    url = f"https://api.helius.xyz/v0/transactions?api-key={helius_api_key}"
    headers = {"Content-Type": "application/json"}

    all_results = []
    batch_size = 100

    for i in range(0, len(signatures), batch_size):
        batch = signatures[i:i+batch_size]
        payload = json.dumps({"transactions": batch})

        response = requests.post(url, headers=headers, data=payload)

        try:
            data = response.json()
            if isinstance(data, dict) and "error" in data:
                print(f"❌ Error at batch {i // batch_size}: {data['error']}")
                continue

            all_results.extend(data)

        except Exception as e:
            print(f"❌ Exception during batch {i // batch_size}: {e}")

        time.sleep(0.3)  # Optional rate limit buffer

    return all_results


In [21]:
# signatures_array

In [22]:
# data = v0_transactions_all(signatures_array, HELIUS_API_KEY)

In [23]:
# len(data)

In [24]:
os.getcwd()

'/Users/olaoluwatunmise/Solana-Forensic-Analytics/Solana-Forensic-Analysis-Tool/notebooks'

In [25]:
# os.chdir('..')

In [26]:
def get_comprehensive_tx_history(wallet, api_key, use_cache=True):

    if use_cache:
        with open('../data/raw/test_tx_history.json') as f:
            parsed_transaction_history = json.load(f)

        return parsed_transaction_history
    
    else:

        signatures = get_all_signatures(wallet, api_key)

        signatures_dict = {}

        for sig in signatures:
            time = sig.get('blockTime')
            signature_str = sig.get('signature')

            signatures_dict[signature_str] = time
        
        signatures_array = list(signatures_dict.keys())

        parsed_transaction_history = v0_transactions_all(signatures_array, api_key)

        with open('../data/raw/test_tx_history.json', 'w') as f:
            json.dump(parsed_transaction_history, f)

        return parsed_transaction_history

In [27]:
parsed_transaction_history = get_comprehensive_tx_history(test_address, HELIUS_API_KEY, use_cache=use_cache)

In [28]:
len(parsed_transaction_history)

493

In [29]:
# os.chdir('notebooks')

In [30]:
os.getcwd()

'/Users/olaoluwatunmise/Solana-Forensic-Analytics/Solana-Forensic-Analysis-Tool/notebooks'

In [31]:
# from helius import TransactionsAPI
# transactions_api  = TransactionsAPI(HELIUS_API_KEY)

In [32]:
# os.getcwd()

In [33]:
# os.chdir('..')

In [34]:
# if not use_cache:
#     parsed_transaction_history = transactions_api.get_parsed_transaction_history(address=test_address)

#     with open('data/test_tx_history.json', 'w') as f:
#         json.dump(parsed_transaction_history, f)
# else:
#     with open('data/test_tx_history.json') as f:
#         parsed_transaction_history = json.load(f)

In [35]:
# os.chdir('notebooks')

In [36]:
# os.getcwd()

In [37]:
# print(json.dumps(parsed_transaction_history, indent=2))

In [38]:
for p in parsed_transaction_history:
    tokenTransfers = p.get('tokenTransfers')
    nativeTransfers = p.get('nativeTransfers')

    print(f'tokenTransfers: {tokenTransfers}, nativeTransfers: {nativeTransfers}')

tokenTransfers: [], nativeTransfers: [{'fromUserAccount': '5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr', 'toUserAccount': 'B1rgyryCqZRazchK9fGifahLqn2LGBzkRByupHW7Dmu8', 'amount': 1}, {'fromUserAccount': '5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr', 'toUserAccount': 'ABXQf1Efhb3PRiJfB3dWs8mPgrHJmLrQBJZVKC1Y3YVj', 'amount': 1}, {'fromUserAccount': '5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr', 'toUserAccount': '9WWzbnzzMBbyUHmPgL3v49D5awe6f6XDFNBLRJ6sZj3s', 'amount': 1}, {'fromUserAccount': '5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr', 'toUserAccount': 'AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU', 'amount': 1}, {'fromUserAccount': '5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr', 'toUserAccount': 'FK5HvFiUyx8x8jnwv2GF3R6of6aeCxgqQtTfHFuqvJGQ', 'amount': 1}, {'fromUserAccount': '5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr', 'toUserAccount': 'CMNncirMrz5yAcSnGrF7GndPj31UinMZMDTa1Swp2Qj7', 'amount': 1}, {'fromUserAccount': '5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr', 'toUserAccou

In [39]:
len(parsed_transaction_history) # each tx is an array

493

In [40]:
parsed_transaction_history[0].keys()

dict_keys(['description', 'type', 'source', 'fee', 'feePayer', 'signature', 'slot', 'timestamp', 'tokenTransfers', 'nativeTransfers', 'accountData', 'transactionError', 'instructions', 'events'])

In [41]:
# TX Id from Flipside dataset matches signatures in Helius dataset 

# signatures = []

# for ar in parsed_transaction_history:
#     signature = ar['signature'].lower()
#     if signature in balance_df['TX_ID'].unique():
#         print(f'ar: {ar}')


In [42]:
#block_time (UTC)

pd.to_datetime(parsed_transaction_history[0].get('timestamp'), unit='s')

Timestamp('2025-04-06 12:12:19')

In [43]:
# Account data

# account_data = {}

# for i in parsed_transaction_history[0].get('accountData'):
#     accounts = i['account']
#     if accounts and test_address in accounts:
#         nativeBalanceChange = i['nativeBalanceChange']
#         tokenBalanceChanges = i['tokenBalanceChanges']

#         account_data['nativeBalanceChange'] = nativeBalanceChange / LAMPORT_SCALE
#         account_data['tokenBalanceChanges'] = tokenBalanceChanges

# account_data

In [44]:
#Instruction data helper function

def get_instruction_data(tx, wallet):
    instructions_data = {}

    for i in tx.get('instructions'):
        accounts = i['accounts']
        if accounts and wallet in accounts:
            data = i['data']
            program_id = i['programId']
            instructions_data['data'] = data
            instructions_data['programId'] = program_id

    return instructions_data

In [45]:
parsed_transaction_history[0].get('instructions')

[{'accounts': [],
  'data': '3dgRf8s6ueV5',
  'programId': 'ComputeBudget111111111111111111111111111111',
  'innerInstructions': []},
 {'accounts': [],
  'data': 'Kv9sKM',
  'programId': 'ComputeBudget111111111111111111111111111111',
  'innerInstructions': []},
 {'accounts': ['5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr',
   'B1rgyryCqZRazchK9fGifahLqn2LGBzkRByupHW7Dmu8'],
  'data': '3Bxs412MvVNQj175',
  'programId': '11111111111111111111111111111111',
  'innerInstructions': []},
 {'accounts': ['5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr',
   'ABXQf1Efhb3PRiJfB3dWs8mPgrHJmLrQBJZVKC1Y3YVj'],
  'data': '3Bxs412MvVNQj175',
  'programId': '11111111111111111111111111111111',
  'innerInstructions': []},
 {'accounts': ['5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr',
   '9WWzbnzzMBbyUHmPgL3v49D5awe6f6XDFNBLRJ6sZj3s'],
  'data': '3Bxs412MvVNQj175',
  'programId': '11111111111111111111111111111111',
  'innerInstructions': []},
 {'accounts': ['5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr',
   

In [46]:
# def summarize_token_transfer(tx, wallet):
#     net_token_flows = defaultdict(float)
#     counterparties = set()

#     for t in tx.get("tokenTransfers", []):
#         mint = t["mint"]
#         amount = float(t["tokenAmount"])  # Already scaled to human-readable units

#         from_user = t.get("fromUserAccount")
#         to_user = t.get("toUserAccount")

#         if from_user == wallet:
#             net_token_flows[mint] -= amount
#             counterparties.add(to_user)
#         elif to_user == wallet:
#             net_token_flows[mint] += amount
#             counterparties.add(from_user)

#     return {
#         "timestamp": tx["timestamp"],
#         "signature": tx["signature"],
#         "net_token_flows": dict(net_token_flows),
#         "counterparties": list(counterparties),
#         "type": tx.get("type", "UNKNOWN"),
#         "source": tx.get("source", "UNKNOWN")
#     }

In [47]:
# token_transfer_data = {}

# for p in parsed_transaction_history:
#     data_dict = summarize_token_transfer(p, test_address)
#     token_transfer_data[data_dict['signature']] = data_dict['net_token_flows']

# token_transfer_data

In [48]:
# token_tx_dict = summarize_token_transfer(parsed_transaction_history[0], test_address)
# token_tx_dict

In [49]:
# def summarize_native_transfer(tx, wallet):
#     net_token_flows = defaultdict(float)
#     counterparties = set()

#     for t in tx.get("nativeTransfers", []):
#         amount = t.get("amount", 0) / LAMPORT_SCALE
#         from_user = t.get("fromUserAccount")
#         to_user = t.get("toUserAccount")

#         if from_user == wallet:
#             net_token_flows["SOL"] -= amount
#             counterparties.add(to_user)
#         elif to_user == wallet:
#             net_token_flows["SOL"] += amount
#             counterparties.add(from_user)

#     return {
#         "timestamp": tx["timestamp"],
#         "signature": tx["signature"],
#         "net_token_flows": dict(net_token_flows),
#         "counterparties": list(counterparties),
#         "type": tx.get("type", "UNKNOWN"),
#         "source": tx.get("source", "UNKNOWN")
#     }


In [50]:
# native_tx_dict = summarize_native_transfer(parsed_transaction_history[0], test_address)
# native_tx_dict

In [51]:
# def summarize_transaction(tx, wallet):
#     net_token_flows = defaultdict(float)
#     counterparties = set()
#     direction = None

#     # Native transfers
#     for t in tx.get("nativeTransfers", []):
#         amount = t.get("amount", 0) / LAMPORT_SCALE
#         from_user = t.get("fromUserAccount")
#         to_user = t.get("toUserAccount")

#         if from_user == wallet:
#             net_token_flows["SOL"] -= amount
#             counterparties.add(to_user)
#             direction = "sent"
#         elif to_user == wallet:
#             net_token_flows["SOL"] += amount
#             counterparties.add(from_user)
#             direction = "received"

#     # Token transfers (already scaled)
#     for t in tx.get("tokenTransfers", []):
#         mint = t.get("mint", "UNKNOWN")
#         try:
#             amount = float(t.get("tokenAmount", 0))
#         except (ValueError, TypeError):
#             amount = 0

#         from_user = t.get("fromUserAccount")
#         to_user = t.get("toUserAccount")

#         if from_user == wallet:
#             net_token_flows[mint] -= amount
#             counterparties.add(to_user)
#             direction = "sent"
#         elif to_user == wallet:
#             net_token_flows[mint] += amount
#             counterparties.add(from_user)
#             direction = "received"

#     return {
#         "timestamp": tx.get("timestamp"),
#         "signature": tx.get("signature"),
#         "net_token_flows": dict(net_token_flows),
#         "counterparties": list(counterparties),
#         "type": tx.get("type", "UNKNOWN"),
#         "source": tx.get("source", "UNKNOWN"),
#         "tx_status": int(tx.get("transactionError") is None),
#         "direction": direction,
#         "counterparty": list(counterparties)[0] if len(counterparties) == 1 else None
#     }

In [52]:
def summarize_transaction(tx, wallet):
    native_amount = 0
    token_address = None
    token_amount = 0
    counterparties = set()
    sender = None
    receiver = None
    direction = None

    # Native transfers (track total SOL involved)
    for t in tx.get("nativeTransfers", []):
        amount = t.get("amount", 0) / LAMPORT_SCALE
        from_user = t.get("fromUserAccount")
        to_user = t.get("toUserAccount")

        if from_user == wallet:
            native_amount -= amount
            counterparties.add(to_user)
            sender = wallet
            receiver = to_user
            direction = "sent"
        elif to_user == wallet:
            native_amount += amount
            counterparties.add(from_user)
            sender = from_user
            receiver = wallet
            direction = "received"

    # Token transfers (assume one token per tx or summarize total of one type)
    for t in tx.get("tokenTransfers", []):
        try:
            amount = float(t.get("tokenAmount", 0))
        except (ValueError, TypeError):
            amount = 0

        mint = t.get("mint", "UNKNOWN")
        from_user = t.get("fromUserAccount")
        to_user = t.get("toUserAccount")

        if from_user == wallet:
            token_address = mint
            token_amount -= amount
            counterparties.add(to_user)
            sender = wallet
            receiver = to_user
            direction = "sent"
        elif to_user == wallet:
            token_address = mint
            token_amount += amount
            counterparties.add(from_user)
            sender = from_user
            receiver = wallet
            direction = "received"

    return {
        "timestamp": tx.get("timestamp"),
        "signature": tx.get("signature"),
        "type": tx.get("type", "UNKNOWN"),
        "source": tx.get("source", "UNKNOWN"),
        "tx_status": int(tx.get("transactionError") is None),
        "direction": direction,
        "sender": sender,
        "receiver": receiver,
        "counterparty": list(counterparties)[0] if len(counterparties) == 1 else None,
        "Native SOL Amount": abs(native_amount) if native_amount else None,
        "token_address": token_address,
        "token_amount": abs(token_amount) if token_address else None
    }

## Transaction-Level Dataset

In [53]:
#Helper function to create row from helius and flipside data

def create_row(tx, wallet, balance_df):
    """
    Takes in a tx from helius data, a wallet address, and balance timeseries data from Flipside.  Returns a df row for analysis 
    """
    tx_summary = summarize_transaction(tx, wallet)
    instructions_data = get_instruction_data(tx, wallet)

    token_tx_df = pd.DataFrame([tx_summary])
    tx_status = "failed" if tx.get("transactionError") else "success"

    token_tx_df['block_number'] = tx.get('slot')
    token_tx_df['tx_fee'] = tx.get('fee') / LAMPORT_SCALE
    token_tx_df['program_id'] = instructions_data.get('programId')
    token_tx_df['tx_status'] = tx_status #if no tx error, can assume it succeeded so we use binary 1 and 0 instead of just leaving nan
    token_tx_df['signature'] = token_tx_df['signature'].str.lower()

    filtered_balance_timeseries = balance_df[['PRE_BALANCE','BALANCE','SYMBOL','NAME','MINT','TX_ID']]
    filtered_balance_timeseries = filtered_balance_timeseries.rename(columns={'TX_ID':'signature'})

    combined_df = pd.merge(token_tx_df,filtered_balance_timeseries,on='signature', how='left')
    return combined_df 



In [54]:
#Testing the workflow to concat the dataset

combined_row = create_row(parsed_transaction_history[0], test_address, balance_df)
combined_row2 = create_row(parsed_transaction_history[1], test_address, balance_df)
pd.concat([combined_row,combined_row2]).drop_duplicates(subset='signature')

,timestamp,signature,type,source,tx_status,direction,sender,receiver,counterparty,Native SOL Amount,token_address,token_amount,block_number,tx_fee,program_id,PRE_BALANCE,BALANCE,SYMBOL,NAME,MINT
0,1743941539,6mpqrks4akyjez3jljjubhswnb96oijdfx1dgeznzebhxm...,TRANSFER,SYSTEM_PROGRAM,success,received,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,1.000000e-09,None,None,331670303,0.000006,11111111111111111111111111111111,0.017773,0.017773,SOL,Solana,So11111111111111111111111111111111111111111
0,1743898250,29ap1yrcx2dbjm4mxk8257g5e87by7cgxysds6liaamsxx...,TRANSFER,SYSTEM_PROGRAM,success,received,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,1.000000e-09,None,None,331561835,0.000005,11111111111111111111111111111111,0.017773,0.017773,SOL,Solana,So11111111111111111111111111111111111111111


In [55]:
tx_level_data = pd.DataFrame()

for i in parsed_transaction_history:
    row = create_row(i, test_address, balance_df)
    tx_level_data = pd.concat([tx_level_data, row]).drop_duplicates(subset='signature')


/var/folders/9f/dspzp56j3pjcftph22hs9gw40000gn/T/ipykernel_99967/1617842129.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tx_level_data = pd.concat([tx_level_data, row]).drop_duplicates(subset='signature')
/var/folders/9f/dspzp56j3pjcftph22hs9gw40000gn/T/ipykernel_99967/1617842129.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tx_level_data = pd.concat([tx_level_data, row]).drop_duplicates(subset='signature')
/var/folders/9f/dspzp56j3pjcftph22hs9gw40000gn/T/ipykernel_99967/1617842129.

In [56]:
tx_level_data['timestamp'] = pd.to_datetime(tx_level_data['timestamp'], unit='s')
tx_level_data.rename(columns={'BALANCE':'POST_BALANCE','NAME':'TOKEN_NAME'},inplace=True)

In [57]:
#Confirming datatypes 

float_cols = ['Native SOL Amount','token_amount', 'tx_fee','PRE_BALANCE', 'POST_BALANCE']
int_cols = ['block_number']
str_cols = ['signature',
       'sender', 'receiver', 'counterparty','SYMBOL', 'TOKEN_NAME', 'MINT']
category_cols = ['type','source','tx_status','direction']

tx_level_data[float_cols] = tx_level_data[float_cols].astype(float)
tx_level_data[int_cols] = tx_level_data[int_cols].astype(float)
tx_level_data[str_cols] = tx_level_data[str_cols].astype(str)
tx_level_data[category_cols] = tx_level_data[category_cols].astype("category")

# Token Prices

In [58]:
tx_level_data['timestamp'].min()

Timestamp('2023-11-14 15:20:07')

In [59]:
cleaned_tokens = tx_level_data[tx_level_data['token_address'].notnull()]['token_address'].unique()
cleaned_tokens

array(['2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo',
       'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB',
       'J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn',
       'So11111111111111111111111111111111111111112',
       'NeonTjSjsuo3rexg9o6vHuMXw62f9V7zvmu8M8Zut44',
       'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
       '2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv',
       'KMNo3nJsBXfcpJTVhZcXLW7RmTwTt4GVFE7suUBo9sS',
       'WENWENvqqNya429ubCdR81ZmD69brwQaaBYY6p3LCpk',
       '9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY',
       '9uvxPXzVYWe8qb7S4gUBqsvASmRMVWo5GSAcMmdZ1LQr',
       'mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So',
       'bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1',
       'B9uEZfxJoAsc51BZs6rNs8XtYVu5vfVrhbCuDCJ7mPEE',
       'BLZEEuZUBVqFhj8adcCFPJvPVCiCyVmh3hkJMrU8KuJA',
       'BgN8VphMQBX4LW6F6sWE6UVuuAmSGcEHPFP6xESG2z1W'], dtype=object)

We will use flipside query to get token prices for each token

https://flipsidecrypto.xyz/studio/queries/9207cf6d-d327-44cf-ab49-c8d6e48e7483

In [60]:
prices_data = pd.read_csv('../data/raw/test_address_prices.csv').dropna()
prices_data['DT'] = pd.to_datetime(pd.to_datetime(prices_data['DT']).dt.strftime('%Y-%m-%d'))
prices_data

,DT,SYMBOL,TOKEN_ADDRESS,PRICE
0,2025-04-16,SOL,So11111111111111111111111111111111111111112,126.960000
1,2025-04-16,$WEN,WENWENvqqNya429ubCdR81ZmD69brwQaaBYY6p3LCpk,0.000029
2,2025-04-16,PENGU,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,0.004451
3,2025-04-16,PYUSD,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,0.999851
4,2025-04-16,MSOL,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,163.086667
...,...,...,...,...
5586,2023-11-15,NEON,NeonTjSjsuo3rexg9o6vHuMXw62f9V7zvmu8M8Zut44,0.389586
5587,2023-11-15,PYUSD,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,0.999079
5588,2023-11-15,BSOL,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,65.818691
5589,2023-11-15,USDC,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,1.000292


In [61]:
def to_time(df):
    time_cols = ['date','dt','hour','time','day','month','year','week','timestamp','date(utc)','block_timestamp']
    for col in df.columns:
        if col.lower() in time_cols and col.lower() != 'timestamp':
            df[col] = pd.to_datetime(df[col])
            df.set_index(col, inplace=True)
        elif col.lower() == 'timestamp':
            df[col] = pd.to_datetime(df[col], unit='ms')
            df.set_index(col, inplace=True)
    print(df.index)
    return df 

def clean_prices(prices_df):
    print('cleaning prices')
    # Pivot the dataframe
    prices_df = prices_df.drop_duplicates(subset=['dt', 'symbol'])
    prices_df_pivot = prices_df.pivot(
        index='dt',
        columns='symbol',
        values='price'
    )
    prices_df_pivot = prices_df_pivot.reset_index()

    # Rename the columns by combining 'symbol' with a suffix
    prices_df_pivot.columns = ['dt'] + [f'{col}_Price' for col in prices_df_pivot.columns[1:]]
    
    print(f'cleaned prices: {prices_df_pivot}')
    return prices_df_pivot

In [62]:
def data_cleaning(df,dropna=True,ffill=False):
    clean_df = clean_prices(df)
    clean_df = to_time(clean_df)
    if dropna == True:
        # clean_df = clean_df.dropna(axis=1, how='any')
        clean_df = clean_df.dropna()
    if ffill == True:
        clean_df = clean_df.resample('d').ffill().bfill()# filling to day

    if '__row_index' in clean_df.columns:
        clean_df.drop(columns=['__row_index'], inplace=True)

    return clean_df

In [63]:
prices_data.columns = prices_data.columns.str.lower()

In [64]:
prices_df = data_cleaning(prices_data,dropna=False,ffill=False)
prices_df

cleaning prices
cleaned prices:             dt  $WEN_Price  BLZE_Price  BSOL_Price  JITOSOL_Price  KMNO_Price  \
0   2023-11-15         NaN    0.000466   65.818691      64.394884         NaN   
1   2023-11-16         NaN    0.000529   69.905150      68.550961         NaN   
2   2023-11-17         NaN    0.000430   63.326877      62.063894         NaN   
3   2023-11-18         NaN    0.000450   62.666044      61.465032         NaN   
4   2023-11-19         NaN    0.000477   64.991365      63.689492         NaN   
..         ...         ...         ...         ...            ...         ...   
514 2025-04-12    0.000024    0.000272  151.734583     150.088333    0.047800   
515 2025-04-13    0.000024    0.000281  156.990000     155.138750    0.050164   
516 2025-04-14    0.000023    0.000279  158.712500     156.805833    0.050176   
517 2025-04-15    0.000024    0.000301  157.542500     155.675417    0.050223   
518 2025-04-16    0.000029    0.000268  153.350000     151.740417    0.049584

,$WEN_Price,BLZE_Price,BSOL_Price,JITOSOL_Price,KMNO_Price,MSOL_Price,NEON_Price,PENGU_Price,PYUSD_Price,SOL_Price,USDC_Price,USDT_Price
dt,,,,,,,,,,,,
2023-11-15,NaN,0.000466,65.818691,64.394884,NaN,69.063271,0.389586,NaN,0.999079,60.344017,1.000292,1.000493
2023-11-16,NaN,0.000529,69.905150,68.550961,NaN,73.483669,0.513279,NaN,0.999018,64.062858,0.999893,1.000274
2023-11-17,NaN,0.000430,63.326877,62.063894,NaN,66.672507,0.529337,NaN,0.999755,58.041037,1.000249,1.000539
2023-11-18,NaN,0.000450,62.666044,61.465032,NaN,65.893903,0.524945,NaN,0.998109,57.515286,1.000058,1.000571
2023-11-19,NaN,0.000477,64.991365,63.689492,NaN,68.432772,0.564577,NaN,0.998897,59.621687,0.999970,1.000635
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-12,0.000024,0.000272,151.734583,150.088333,0.047800,161.268333,0.115673,0.005121,0.999708,125.725833,0.999952,0.999646
2025-04-13,0.000024,0.000281,156.990000,155.138750,0.050164,166.918750,0.120946,0.005388,0.999770,130.178750,0.999943,0.999769
2025-04-14,0.000023,0.000279,158.712500,156.805833,0.050176,168.759167,0.114790,0.004997,0.999932,131.470417,0.999946,0.999813


In [65]:
tx_level_data['day'] = pd.to_datetime(tx_level_data['timestamp'].dt.strftime('%Y-%m-%d'))
tx_level_data

,timestamp,signature,type,source,tx_status,direction,sender,receiver,counterparty,Native SOL Amount,...,token_amount,block_number,tx_fee,program_id,PRE_BALANCE,POST_BALANCE,SYMBOL,TOKEN_NAME,MINT,day
0,2025-04-06 12:12:19,6mpqrks4akyjez3jljjubhswnb96oijdfx1dgeznzebhxm...,TRANSFER,SYSTEM_PROGRAM,success,received,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,1.000000e-09,...,NaN,331670303.0,0.000006,11111111111111111111111111111111,0.017773,0.017773,SOL,Solana,So11111111111111111111111111111111111111111,2025-04-06
0,2025-04-06 00:10:50,29ap1yrcx2dbjm4mxk8257g5e87by7cgxysds6liaamsxx...,TRANSFER,SYSTEM_PROGRAM,success,received,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,1.000000e-09,...,NaN,331561835.0,0.000005,11111111111111111111111111111111,0.017773,0.017773,SOL,Solana,So11111111111111111111111111111111111111111,2025-04-06
0,2025-04-06 00:10:45,2d7d4ndxhzfwz6trn6pzy7pqefkrktqhqjq1zpz5myndvs...,TRANSFER,SYSTEM_PROGRAM,success,received,FLiPGqowc82LLR173hKiFYBq2fCxLZEST5iHbHwj8xKb,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,FLiPGqowc82LLR173hKiFYBq2fCxLZEST5iHbHwj8xKb,1.000000e-09,...,NaN,331561822.0,0.000005,11111111111111111111111111111111,0.017773,0.017773,SOL,Solana,So11111111111111111111111111111111111111111,2025-04-06
0,2025-04-05 23:38:43,nprzwng7slnepcpebwjemkadd6wnuwscrgeweqsnshiznn...,TRANSFER,SYSTEM_PROGRAM,success,received,6UgXZZBoydXRNX6SLjwFDUUgn19GhjBpvNAXU3iepzUV,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,6UgXZZBoydXRNX6SLjwFDUUgn19GhjBpvNAXU3iepzUV,1.000000e-05,...,NaN,331557009.0,0.000005,11111111111111111111111111111111,0.017763,0.017773,SOL,Solana,So11111111111111111111111111111111111111111,2025-04-05
0,2025-04-05 23:38:25,yqsamwung3pc9h5ezjmtr1mudcuuajzye13bmfas2atlvc...,TRANSFER,SYSTEM_PROGRAM,success,received,GUq7PhyAUZko2mPhv3CupmdJKQ61LH8VyrdsRL25q7zg,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,GUq7PhyAUZko2mPhv3CupmdJKQ61LH8VyrdsRL25q7zg,2.000000e-09,...,NaN,331556964.0,0.000005,11111111111111111111111111111111,0.017763,0.017763,SOL,Solana,So11111111111111111111111111111111111111111,2025-04-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2023-11-14 15:44:24,2slwlw7rkrw4xsghxjaxmqidy2yaabjstn6u1h7hekud4d...,TRANSFER,SYSTEM_PROGRAM,success,received,FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7,NaN,...,NaN,229991035.0,0.000005,11111111111111111111111111111111,NaN,NaN,nan,nan,nan,2023-11-14
0,2023-11-14 15:43:54,3mwzl67964c2emlndhj37zqyjq6g2vfjqv1asfyhvednzi...,TRANSFER,SYSTEM_PROGRAM,success,received,FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7,NaN,...,NaN,229990970.0,0.000005,11111111111111111111111111111111,NaN,NaN,nan,nan,nan,2023-11-14
0,2023-11-14 15:43:23,2iqht33xr4mhcrkvrttzkqgcw16m8usvvv2xhxm2zgcgch...,SWAP,JUPITER,success,received,6LXutJvKUw8Q5ue2gCgKHQdAN4suWW8awzFVC6XCguFx,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,None,2.369678e-01,...,12.927220,229990897.0,0.000005,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.469857,0.232884,SOL,Solana,So11111111111111111111111111111111111111111,2023-11-14
0,2023-11-14 15:25:34,yvctdnezqofcvzv66mkasuuwk3cdz63wshxtshtuafxn52...,TOKEN_MINT,UNKNOWN,failed,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,BQ72nSv9f3PRyRKCBnHLVrerrv37CYTHm5h3s9VSGQDV,None,2.369704e-01,...,0.234931,229988531.0,0.000005,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.469862,0.469857,SOL,Solana,So11111111111111111111111111111111111111111,2023-11-14


In [66]:
# Ensure datetime columns are aligned and named the same for merging
prices_data = prices_data.rename(columns={'DT': 'day', 'TOKEN_ADDRESS': 'MINT'})
prices_data['day'] = pd.to_datetime(prices_data['day'])

# Merge the price data into the tx_level_data
merged_df = tx_level_data.merge(prices_data[['day', 'MINT', 'PRICE']], on=['day', 'MINT'], how='left')

# Create the token_amount_usd column
merged_df['token_amount_usd'] = merged_df['token_amount'] * merged_df['PRICE']


KeyError: 'day'

## We need to streamline which token is being used for calculation

In [ ]:
merged_df[merged_df['token_amount_usd']>0][['token_amount','PRICE','token_amount_usd','POST_BALANCE','PRE_BALANCE']]

,token_amount,PRICE,token_amount_usd,POST_BALANCE,PRE_BALANCE
6,128.319372,0.999819,128.296130,0.000000,128.319372
7,1.180087,0.999645,1.179669,0.000000,129.499459
8,129.499459,0.999645,129.453546,129.499459,0.000000
17,493.070747,0.999808,492.976077,0.000000,493.070747
20,4.377959,0.999939,4.377694,0.000000,497.448706
...,...,...,...,...,...
468,195.106115,0.001642,0.320436,195.106115,0.000000
475,23.467897,1.000075,23.469653,23.470387,0.002490
479,15.520000,0.999970,15.519530,0.002490,15.522490
480,15.000000,0.999970,14.999546,15.522490,30.522490


# Saving the Dataset

In [ ]:
tx_level_data.to_csv('../data/processed/test_acc_timeseries.csv', index=False)

In [ ]:
tx_level_data

%store tx_level_data

Stored 'tx_level_data' (DataFrame)


# Nansen Data

In [ ]:
# I think the API is not public yet

# Misc

In [ ]:
# # helpers/utility.py
# def _make_post_request(url: str, payload: dict, headers: dict) -> dict:
#     response = requests.post(url, json=payload, headers=headers)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         raise ValueError(f"Error: {response.status_code}: {response.text}")

In [ ]:
# # Initialize with your API key (ensure it's valid and active)
# #HELIUS_API_KEY = os.getenv("api_key")  
# tx_api = TransactionsAPI(api_key=HELIUS_API_KEY)

# tx_api

In [ ]:
# # Example usage
# transactions = [""]  # Your transaction signatures
# response = tx_api.get_parsed_transactions(transactions)
# print(response)

In [ ]:
balances_api = BalancesAPI(HELIUS_API_KEY)
balances_api

In [ ]:
# balances_api.get_balances("hq8fC9VkxU6rv1QvMUkGKbhFR3MxSd7nCkCGaaXx9MA")